# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0830 11:04:45.562000 4135796 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:04:45.562000 4135796 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:04:46] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32432, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=404706561, constrained_

[2025-08-30 11:04:46] Using default HuggingFace chat template with detected content format: string


W0830 11:04:53.656000 4136829 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:04:53.656000 4136829 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0830 11:04:53.656000 4136830 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:04:53.656000 4136830 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:04:54] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-30 11:04:54] Init torch distributed begin.


...

Capturing batches (bs=3 avail_mem=53.68 GB):  25%|██▌       | 1/4 [00:18<00:55, 18.55s/it]

Capturing batches (bs=1 avail_mem=53.64 GB):  50%|█████     | 2/4 [00:19<00:16,  8.10s/it]

Capturing batches (bs=1 avail_mem=53.64 GB): 100%|██████████| 4/4 [00:23<00:00,  5.80s/it]
[2025-08-30 11:05:56] Capture draft cuda graph end. Time elapsed: 23.71 s. mem usage=0.13 GB. avail mem=38.02 GB.
[2025-08-30 11:05:56] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=38.02 GB


Capturing batches (bs=1 avail_mem=37.94 GB): 100%|██████████| 4/4 [00:00<00:00, 115.49it/s]
[2025-08-30 11:05:57] Capture draft extend cuda graph end. Time elapsed: 0.61 s. mem usage=0.08 GB. avail mem=37.94 GB.
[2025-08-30 11:05:57] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=37.94 GB
[2025-08-30 11:05:57] INFO:     Started server process [4135796]
[2025-08-30 11:05:57] INFO:     Waiting for application startup.
[2025-08-30 11:05:57] INFO:     Application startup complete.
[2025-08-30 11:05:57] INFO:     Uvicorn running on http://127.0.0.1:32432 (Press CTRL+C to quit)


[2025-08-30 11:05:58] INFO:     127.0.0.1:45564 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-30 11:05:58] INFO:     127.0.0.1:45566 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-30 11:05:58] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-30 11:06:49] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-30 11:06:49] INFO:     127.0.0.1:45576 - "POST /generate HTTP/1.1" 200 OK
[2025-08-30 11:06:49] The server is fired up and ready to roll!


[2025-08-30 11:06:50] INFO:     127.0.0.1:45584 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0830 11:06:59.012000 4144911 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:06:59.012000 4144911 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:06:59] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=31174, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=549621008, constrained_js

[2025-08-30 11:07:00] Using default HuggingFace chat template with detected content format: string


W0830 11:07:07.473000 4145467 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:07:07.473000 4145467 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0830 11:07:07.479000 4145468 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:07:07.479000 4145468 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:07:08] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-30 11:07:08] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=5.01 GB): 100%|██████████| 4/4 [00:16<00:00,  4.19s/it]
[2025-08-30 11:08:12] Capture draft cuda graph end. Time elapsed: 17.45 s. mem usage=33.55 GB. avail mem=3.95 GB.
[2025-08-30 11:08:12] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=3.95 GB


Capturing batches (bs=1 avail_mem=3.85 GB): 100%|██████████| 4/4 [00:00<00:00, 37.31it/s]
[2025-08-30 11:08:13] Capture draft extend cuda graph end. Time elapsed: 0.98 s. mem usage=0.10 GB. avail mem=3.85 GB.
[2025-08-30 11:08:13] max_total_num_tokens=18695, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=3.85 GB


[2025-08-30 11:08:13] INFO:     Started server process [4144911]
[2025-08-30 11:08:13] INFO:     Waiting for application startup.
[2025-08-30 11:08:13] INFO:     Application startup complete.
[2025-08-30 11:08:13] INFO:     Uvicorn running on http://127.0.0.1:31174 (Press CTRL+C to quit)
[2025-08-30 11:08:13] INFO:     127.0.0.1:34730 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-30 11:08:14] INFO:     127.0.0.1:34744 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-30 11:08:14] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-30 11:08:16] INFO:     127.0.0.1:34750 - "POST /generate HTTP/1.1" 200 OK
[2025-08-30 11:08:16] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-30 11:08:18] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-30 11:08:19] INFO:     127.0.0.1:42440 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0830 11:08:26.186000 4153962 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:08:26.186000 4153962 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:08:26] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=33013, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=434542418,

[2025-08-30 11:08:27] Using default HuggingFace chat template with detected content format: string


W0830 11:08:34.486000 4154176 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:08:34.486000 4154176 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0830 11:08:34.529000 4154177 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:08:34.529000 4154177 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:08:34] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=56.15 GB): 100%|██████████| 4/4 [00:06<00:00,  1.70s/it]
[2025-08-30 11:09:03] Capture draft cuda graph end. Time elapsed: 7.38 s. mem usage=-16.29 GB. avail mem=56.11 GB.
[2025-08-30 11:09:03] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=56.11 GB


Capturing batches (bs=1 avail_mem=56.01 GB): 100%|██████████| 4/4 [00:00<00:00, 87.10it/s]
[2025-08-30 11:09:04] Capture draft extend cuda graph end. Time elapsed: 0.65 s. mem usage=0.10 GB. avail mem=56.01 GB.
[2025-08-30 11:09:04] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=56.01 GB


[2025-08-30 11:09:04] INFO:     Started server process [4153962]
[2025-08-30 11:09:04] INFO:     Waiting for application startup.
[2025-08-30 11:09:04] INFO:     Application startup complete.
[2025-08-30 11:09:04] INFO:     Uvicorn running on http://127.0.0.1:33013 (Press CTRL+C to quit)


[2025-08-30 11:09:05] INFO:     127.0.0.1:52474 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-30 11:09:05] INFO:     127.0.0.1:32778 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-30 11:09:05] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-30 11:09:06] INFO:     127.0.0.1:32782 - "POST /generate HTTP/1.1" 200 OK
[2025-08-30 11:09:06] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-30 11:09:10] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-30 11:09:10] INFO:     127.0.0.1:32798 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0830 11:09:18.102000 4157239 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:09:18.102000 4157239 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:09:19] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34823, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=347225172, const

[2025-08-30 11:09:19] Using default HuggingFace chat template with detected content format: string


W0830 11:09:26.818000 4158196 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:09:26.818000 4158196 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0830 11:09:26.880000 4158197 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:09:26.880000 4158197 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:09:27] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=56.68 GB): 100%|██████████| 4/4 [00:00<00:00, 39.09it/s]
[2025-08-30 11:09:55] Capture draft extend cuda graph end. Time elapsed: 1.42 s. mem usage=1.11 GB. avail mem=56.58 GB.
[2025-08-30 11:09:55] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=56.58 GB


[2025-08-30 11:09:56] INFO:     Started server process [4157239]
[2025-08-30 11:09:56] INFO:     Waiting for application startup.
[2025-08-30 11:09:56] INFO:     Application startup complete.
[2025-08-30 11:09:56] INFO:     Uvicorn running on http://127.0.0.1:34823 (Press CTRL+C to quit)


[2025-08-30 11:09:56] INFO:     127.0.0.1:54978 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-30 11:09:57] INFO:     127.0.0.1:54990 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-30 11:09:57] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-30 11:09:57] INFO:     127.0.0.1:55002 - "POST /generate HTTP/1.1" 200 OK
[2025-08-30 11:09:57] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-30 11:10:01] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-30 11:10:02] INFO:     127.0.0.1:55008 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0830 11:10:09.098000 4162125 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:10:09.098000 4162125 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:10:10] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=35686, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=220034123, constrained_json_whitespace_patte

[2025-08-30 11:10:10] Using default HuggingFace chat template with detected content format: string


W0830 11:10:19.180000 4162607 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:10:19.180000 4162607 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0830 11:10:19.207000 4162606 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0830 11:10:19.207000 4162606 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-30 11:10:20] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-30 11:10:20] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=41.48 GB):  50%|█████     | 2/4 [00:01<00:01,  1.49it/s]

Capturing batches (bs=1 avail_mem=41.48 GB): 100%|██████████| 4/4 [00:01<00:00,  2.07it/s]
[2025-08-30 11:10:30] Capture draft cuda graph end. Time elapsed: 2.42 s. mem usage=0.01 GB. avail mem=41.48 GB.
[2025-08-30 11:10:30] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=41.48 GB


Capturing batches (bs=1 avail_mem=41.38 GB): 100%|██████████| 4/4 [00:00<00:00, 50.33it/s]
[2025-08-30 11:10:30] Capture draft extend cuda graph end. Time elapsed: 0.58 s. mem usage=0.09 GB. avail mem=41.38 GB.
[2025-08-30 11:10:30] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=41.38 GB


[2025-08-30 11:10:31] INFO:     Started server process [4162125]
[2025-08-30 11:10:31] INFO:     Waiting for application startup.
[2025-08-30 11:10:31] INFO:     Application startup complete.
[2025-08-30 11:10:31] INFO:     Uvicorn running on http://0.0.0.0:35686 (Press CTRL+C to quit)


[2025-08-30 11:10:32] INFO:     127.0.0.1:41708 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-30 11:10:32] INFO:     127.0.0.1:41712 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-30 11:10:32] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-30 11:10:32] INFO:     127.0.0.1:41718 - "POST /generate HTTP/1.1" 200 OK
[2025-08-30 11:10:32] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-30 11:10:37] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-30 11:10:37] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: True, gen throughput (token/s): 10.55, #queue-req: 0, 


[2025-08-30 11:10:37] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, accept len: 1.75, cuda graph: True, gen throughput (token/s): 180.00, #queue-req: 0, 


[2025-08-30 11:10:38] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, accept len: 1.80, cuda graph: True, gen throughput (token/s): 181.71, #queue-req: 0, 
[2025-08-30 11:10:38] INFO:     127.0.0.1:56204 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).